In [1]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

In [2]:
env = gym.make('CartPole-v1', render_mode="human")

episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    terminated = False
    truncated = False
    score = 0

    while not terminated and not truncated:
        env.render()
        action = env.action_space.sample() # Action can be 0 or 1
        n_obs, reward, terminated, truncated, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

env.close()

Episode:1 Score:14.0
Episode:2 Score:33.0
Episode:3 Score:15.0
Episode:4 Score:32.0
Episode:5 Score:13.0


In [3]:
# Display action and observation spaces
print(env.action_space)
print('An example of an action:', env.action_space.sample())

print(env.observation_space)
print('Cart position:', env.observation_space.sample()[0])
print('Cart velocity:', env.observation_space.sample()[1])
print('Pole angle:', env.observation_space.sample()[2])
print('Pole velocity:', env.observation_space.sample()[3])


Discrete(2)
An example of an action: 0
Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Cart position: -0.6193639
Cart velocity: 1.2982815e+37
Pole angle: -0.33000752
Pole velocity: -3.0405368e+37


In [4]:
log_path = os.path.join('Training', 'Logs')

# Parallel environments
env = make_vec_env("CartPole-v1", n_envs=1)
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

# Train the agent
model.learn(total_timesteps=30000)

Using cpu device
Logging to Training\Logs\PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.9     |
|    ep_rew_mean     | 21.9     |
| time/              |          |
|    fps             | 254      |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 26.3        |
|    ep_rew_mean          | 26.3        |
| time/                   |             |
|    fps                  | 366         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009119647 |
|    clip_fraction        | 0.125       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.685      |
|    explained_variance 

In [5]:
# Save the agent
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_CartPole')
model.save(PPO_path)

In [6]:
del model # remove to demonstrate saving and loading

# Load the trained agent
model = PPO.load(PPO_path, env=env)

In [7]:
# Evaluate the agent
evaluate_policy(model, env, n_eval_episodes=10)


(500.0, 0.0)

In [8]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

env.close()

Episode:1 Score:[500.]
Episode:2 Score:[500.]
Episode:3 Score:[500.]
Episode:4 Score:[500.]
Episode:5 Score:[500.]


### Logs

To view logs for training, run the following command in the terminal:

```
tensorboard --logdir='Logs<your_log_dir>'
```

In [12]:
# Adding a callback
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

save_path = os.path.join('Training', 'Saved Models')

stop_callback = StopTrainingOnRewardThreshold(reward_threshold=500, verbose=1)
eval_callback = EvalCallback(env, 
                            callback_on_new_best=stop_callback, 
                            eval_freq=10000,
                            best_model_save_path=save_path,
                            verbose=1)

In [13]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)
model.learn(total_timesteps=20000, callback=eval_callback)

Using cpu device
Logging to Training\Logs\PPO_5
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 23.5     |
|    ep_rew_mean     | 23.5     |
| time/              |          |
|    fps             | 966      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 25.5       |
|    ep_rew_mean          | 25.5       |
| time/                   |            |
|    fps                  | 824        |
|    iterations           | 2          |
|    time_elapsed         | 4          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00832076 |
|    clip_fraction        | 0.0975     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.686     |
|    explained_variance   | 0.00544   

In [15]:
# Changing Policies
net_architecture = [dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

model = PPO("MlpPolicy", env, verbose=1, policy_kwargs=dict(net_arch=net_architecture), tensorboard_log=log_path)

Using cpu device


In [16]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_6
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.4     |
|    ep_rew_mean     | 21.4     |
| time/              |          |
|    fps             | 936      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 28.6        |
|    ep_rew_mean          | 28.6        |
| time/                   |             |
|    fps                  | 618         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015157977 |
|    clip_fraction        | 0.234       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | 0.00245     |

In [18]:
# Alternative Algorithms
from stable_baselines3 import DQN

model = DQN("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

model.learn(total_timesteps=20000, callback=eval_callback)

Using cpu device
Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 26       |
|    ep_rew_mean      | 26       |
|    exploration_rate | 0.951    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2152     |
|    time_elapsed     | 0        |
|    total_timesteps  | 104      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 25.1     |
|    ep_rew_mean      | 25.1     |
|    exploration_rate | 0.905    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 2782     |
|    time_elapsed     | 0        |
|    total_timesteps  | 201      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 24       |
|    ep_rew_mean      | 24       |
|    exploration_rate | 0.863    |
| time/